
# 정규식 생성기 Regular Expression Generator 

Haskell로 정규식을 생성하는 프로그램을 3단계에 걸쳐 작성해 나가는 예제를 소개한다.
2018년 2학기 관련 과목 강의들의 초반부에 이 예제를 시작으로 하여 함수형 프로그래밍과 정규식을 동시에 학습해서 시간을 절약하는 것이 목표이다.

## 정규식 문법을 하스켈 데이타 타입으로 정의하기

정규식의 문법은 다음과 같이 정의된다.
$$
\begin{array}{l}
c \in \Sigma \\
R ::= \varnothing \mid \varepsilon \mid c \mid R \cdot R \mid R + R \mid R{*}
\end{array}
$$

우선 언어를 구성하는 가장 기본적 단위인 글자의 범위를 정해야 하는데 바로 $\Sigma$가 언어를 구성하는 알파벳의 집합이다. 예를 들면 영어 표기를 구성하는 $\Sigma=\{a,b,c,\ldots,x,y,z\}$라고 대략 비유할 수 있다. 영어는 정규언어도 아니고 더구나 형식언어가 아닌 자연언어이기 때문에 물론 그냥 비유일 뿐이다. 컴퓨터 과학에서 다루는 형식언어 중 구조가 가장 단순한 저급언어인 기계어 등의 이진수로 이루어진 언어들의 경우 $\Sigma=\{0,1\}$이 된다. 위에서는 $\Sigma$의 원소인 글자를 $c$로 표기했다.

정규식 $R$을 구성하는 가장 기초적인 방법은 $\varnothing$, $\varepsilon$, $c$ 이렇게 세 가지이며,
기존의 정규식으로부터 새로운 정규식을 구성하는 방법도 이항연산자인 $\cdot$과 $+$ 그리고 단항연산자인 $*$를 이용하는 세 가지이다.
편의상 $\cdot$은 생략하기도 한다. 즉, $c_1 \cdot c_2$라고 쓰는 대신 $c_1c_2$라고 간단히 쓰기도 한다.

이렇게 정규식을 구성하는 여섯 가지 방법을 하스켈 데이타 타입으로 옮겨서 정의하면 아래와 같다.

In [1]:
data RE -- 정규식 데이타 타입
  = Empty
  | Epsilon
  | Alphabet Char
  | Concat RE RE
  | Union RE RE
  | Kleene RE
  deriving Show

엄밀히 말하자면 수학적 정의보다는 좀 느슨하게 데이타 타입을 정의했다. 구체적으로 느슨한 부분을 지적하자면 세번째 경우에 해당하는 `Alphabet Char`이다. 수학적 정의를 그대로 따르자면 일단 $\Sigma$에 해당하는 타입을 예컨데 `Sigma`라는 하스켈 데이타 타입을 먼저 정의하거나 `RE`를 타입 인자를 받는 타입 생성자로 만들어 `Sigma` 파라메터로 한다던가 하고 `Alphabet Sigma`라고 하면 수식 표현으로 정의한 것과 더 일치할 것이다.

위 `RE` 데이타타입 정의에서는 편의상 하스켈에서 지원하는 문자를 그대로 그냥 활용하기 위해 하스켈에서 제공하는 문자 타입인 `Char`를 썼다.

즉 $\Sigma$를 따로 정의하거나 타입 파라메터로 일반화하는 대신
$\Sigma$를 하스켈 문자 타입인 `Char`로 고정해 버린 것이다.

프로그래밍 언어에서 값을 분류하는 개념이 타입이다.
그리고 그러한 타입 및 타입 생성자를 분류하는 개념을 kind라고 하며 아래와 같이 `:kind` 명령어로 타입 및 타입 생성자의 kind를 알아볼 수 있다.

In [2]:
:kind RE -- 앞서 정의한 정규식 문법
:kind Int -- 하스켈에서 제공하는 범위가 유한한 정수 타입
:kind [] -- 리스트 타입 생성자 type constuctor
:kind [] Int -- 정수 리스트 타입; 일반적인 전위(prefix) 표기법으로 []를 Int에 적용
:kind [Int] -- 정수 리스트 타입; [] 타입 생성자를 위한 특별한 표기법을 지원한다

RE :: *

Int :: *

[] :: * -> *

[] Int :: *

[Int] :: *

정의된 데이타 생성자의 타입을 알아보려면 다음과 같이 하면 된다.
다른 값이나 식들도 마찬가지로 `:type` 명령어를 통해 알아볼 수 있다.

In [3]:
:type Empty
:type Epsilon
:type Alphabet
:type Concat
:type Union
:type Kleene

Empty :: RE

Epsilon :: RE

Alphabet :: Char -> RE

Concat :: RE -> RE -> RE

Union :: RE -> RE -> RE

Kleene :: RE -> RE

formatRE :: RE -> [Char]

In [10]:
import IHaskell.Display

ppRE r = Display [html(formatRE r)]

formatRE Empty = "∅"
formatRE Epsilon = "ε"
formatRE (Alphabet c) = c:[]
formatRE (Concat r1 r2) = formatRE r1 ++ formatRE r2
formatRE (Union r1 r2) = "(" ++ formatRE r1 ++ "+" ++ formatRE r2 ++ ")"
formatRE (Kleene r) = "(" ++ formatRE r ++ ")*"

IHaskell 환경에서는 유니코드가 그냥 터미널에서 출력하는 것처럼 되지 않아서
IHaskell에서 지원하는 HTML 출력을 통해 유니코드 글자를 출력한다.

`formatRE`는 하스켈에서 정의한 정규식 타입(RE)의 값을 유니코드 문자를 포함한 수식 표현에 가까운 문자열로 변환하는 함수이다. `ppRE`는 `formatRE`로 변환된 문자열을 IHaskell의 Dispaly 모듈에서 제공하는 HTML 출력 기능을 이용해 노트북에 나타내주기 위한 함수이다.

In [11]:
genRE :: RE -> [String]
genRE Empty          = []
genRE Epsilon        = [ "" ]
genRE (Alphabet c)   = [ c:"" ]
genRE (Concat r1 r2) = [s1++s2 | s1<-genRE r1, s2<-genRE r2]
genRE (Union r1 r2)  = genRE r1 ++ genRE r2
genRE (Kleene r)     = genRE (Union Epsilon (Concat r (Kleene r)))

-- 문자열을 Concat으로 이어진 정규식으로 변환해주는 유틸리티 함수
string2re :: String -> RE
string2re "" = Epsilon
string2re s  = foldr1 Concat (map Alphabet s)

In [5]:
_1 = Alphabet '1'
_0 = Alphabet '0'
_00 = string2re "00"
_01 = string2re "01"
_10 = string2re "10"
_11 = string2re "11"

ppRE Empty
genRE Empty

ppRE Epsilon
genRE Epsilon

ppRE _0
genRE _0

ppRE _1
genRE _1

ppRE (Concat _0 _1)
genRE (Concat _0 _1)

ppRE (Union _0 _1)
genRE (Union _0 _1)

ppRE (Union _00 _11)
genRE (Union _00 _11)

ppRE (Concat (Union _00 _11) (Union _01 _10))
genRE (Concat (Union _00 _11) (Union _01 _10))

ppRE (Kleene _1)
take 10 $ genRE (Kleene _1)

ppRE (Kleene _01)
take 10 $ genRE (Kleene _01)

ppRE (Union (Kleene _0) (Kleene _1))
take 10 $ genRE (Union (Kleene _0) (Kleene _1)) -- 0, 00, 000 같은 것만 나오고 1이 들어가는 건 안나타남

ppRE (Kleene (Union _00 _11))
take 10 $ genRE (Kleene (Union _00 _11)) -- 00, 0000 같은 것만 나오고 11이 들어가는 건 안나타남

∅

[]

ε

[""]

0

...

["","01","0101","010101","01010101","0101010101","010101010101","01010101010101","0101010101010101","010101010101010101"]

((0)*+(1)*)

["","0","00","000","0000","00000","000000","0000000","00000000","000000000"]

((00+11))*

["","00","0000","000000","00000000","0000000000","000000000000","00000000000000","0000000000000000","000000000000000000"]

In [6]:
merge [] ys = ys
merge (x:xs) ys = x:merge ys xs

genRE' Empty          = []
genRE' Epsilon        = [ "" ]
genRE' (Alphabet c)   = [ c:"" ]
genRE' (Concat r1 r2) = [s1++s2 | s1<-genRE' r1, s2<-genRE' r2]
genRE' (Union r1 r2)  = merge (genRE' r1) (genRE' r2)
genRE' (Kleene r)     = genRE' (Union Epsilon (Concat r (Kleene r)))

In [7]:
genRE' Empty
genRE' Epsilon
genRE' _0
genRE' _1
genRE' (Concat _0 _1)
genRE' (Union _0 _1)
genRE' (Union _00 _11)
genRE' (Concat (Union _00 _11) (Union _01 _10))
take 10 $ genRE' (Kleene _1)
take 10 $ genRE' (Kleene _01)
take 10 $ genRE' (Union (Kleene _0) (Kleene _1))
take 10 $ genRE' (Kleene (Union _00 _11))  -- 00, 0000 같은 것만 나오고 11이 들어가는 건 안나타남

[]

[""]

["0"]

["1"]

["01"]

...

["0001","0010","1101","1110"]

["","1","11","111","1111","11111","111111","1111111","11111111","111111111"]

["","01","0101","010101","01010101","0101010101","010101010101","01010101010101","0101010101010101","010101010101010101"]

["","","0","1","00","11","000","111","0000","1111"]

["","00","0000","000000","00000000","0000000000","000000000000","00000000000000","0000000000000000","000000000000000000"]

In [8]:
replicateRE r 0 = Epsilon
replicateRE r n = foldr1 Concat (replicate n r)

genRE'' Empty          = []
genRE'' Epsilon        = [ "" ]
genRE'' (Alphabet c)   = [ c:"" ]
genRE'' (Concat r1 r2) = [s1++s2 | s1<-genRE'' r1, s2<-genRE'' r2]
genRE'' (Union r1 r2)  = merge (genRE'' r1) (genRE'' r2)
genRE'' (Kleene r)     = concatMap (genRE'' . replicateRE r) [0..]

In [9]:
genRE'' Empty
genRE'' Epsilon
genRE'' _0
genRE'' _1
genRE'' (Concat _0 _1)
genRE'' (Union _0 _1)
genRE'' (Union _00 _11)
genRE'' (Concat (Union _00 _11) (Union _01 _10))
take 10 $ genRE'' (Kleene _1)
take 10 $ genRE'' (Kleene _01)
take 10 $ genRE'' (Union (Kleene _0) (Kleene _1))
take 10 $ genRE'' (Kleene (Union _00 _11))

[]

[""]

["0"]

["1"]

["01"]

...

["0001","0010","1101","1110"]

["","1","11","111","1111","11111","111111","1111111","11111111","111111111"]

["","01","0101","010101","01010101","0101010101","010101010101","01010101010101","0101010101010101","010101010101010101"]

["","","0","1","00","11","000","111","0000","1111"]

["","00","11","0000","0011","1100","1111","000000","000011","001100"]